# Leveraged ETF Strategy Execution

Manages the **TECL + NAIL (50/50)** leveraged momentum strategy.

| Step | Description |
|------|-------------|
| 1 | Setup & configuration |
| 2 | Detect current strategy state (IB positions) |
| 3 | Compute live signals (SMA 200 + vol filter) |
| 4 | Generate trade plan |
| 5 | Execute trades (CONFIRM=True to go live) |
| 6 | Check order status |
| 7 | Fix unfilled orders |

**Strategy:** Config C (SMA 200 + vol filter, no trailing stops, no bonds)  
**Exit signal:** SMA 200 crossover (NOT trailing stops — stops destroyed leveraged ETF performance)  
**Reference:** LEVERAGED_STRATEGY_DOCUMENT.tex v3.0

---
## Step 1: Setup & Configuration

In [8]:
import sys
from pathlib import Path

import nest_asyncio
import pandas as pd
nest_asyncio.apply()

PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT / "src"))
sys.path.insert(0, str(PROJECT_ROOT / "notebooks"))

DATA_DIR = Path.home() / "trade_data" / "ETFTrader" / "ib_historical"
LIVE_DIR = Path.home() / "trading" / "leveraged_portfolio"
LIVE_DIR.mkdir(parents=True, exist_ok=True)

# ── IB Settings ────────────────────────────────────────
IB_HOST = "127.0.0.1"
IB_PORT = 4001
IB_CLIENT_ID = 17

# ── Strategy Parameters (Config C: SMA 200 + vol filter) ──
BUDGET = 30_000                    # Initial cash allocation
TICKERS = ["TECL", "NAIL"]         # 50/50 equal weight
REFERENCES = {"TECL": "XLK", "NAIL": "ITB"}
VOL_THRESHOLDS = {"XLK": 0.22, "ITB": 0.22}
TARGET_WEIGHTS = {"TECL": 0.50, "NAIL": 0.50}

# ── Order Settings ─────────────────────────────────────
USE_LIMIT_ORDERS = True            # LIMIT GTC for outside-hours
LIMIT_BUFFER_PCT = 1.0             # 1% buffer from last price

print(f"Budget:     ${BUDGET:,}")
print(f"Tickers:    {TICKERS}")
print(f"References: {REFERENCES}")
print(f"Vol thresh: {VOL_THRESHOLDS}")
print(f"Order mode: {'LIMIT GTC' if USE_LIMIT_ORDERS else 'MARKET'}")
print(f"Output:     {LIVE_DIR}")

Budget:     $30,000
Tickers:    ['TECL', 'NAIL']
References: {'TECL': 'XLK', 'NAIL': 'ITB'}
Vol thresh: {'XLK': 0.22, 'ITB': 0.22}
Order mode: LIMIT GTC
Output:     /home/stuar/trading/leveraged_portfolio


---
## Step 2: Detect Current Strategy State

Connects read-only to IB to check if TECL/NAIL positions exist.

In [9]:
from scripts.s12_leveraged_execute import detect_strategy_state

state = detect_strategy_state(
    tickers=TICKERS,
    ib_host=IB_HOST, ib_port=IB_PORT, ib_client_id=IB_CLIENT_ID,
)

Strategy NOT running — no positions in TECL or NAIL
  Account cash: $106,754


---
## Step 3: Compute Live Signals

Loads reference index prices (XLK, ITB) from IB cache and computes:
- **SMA 200**: Is the reference above its 200-day moving average?
- **Vol filter**: Is the 20-day realised vol below the threshold (22%)?
- **Signal state**: RISK_ON (both conditions met) or RISK_OFF

**RISK_ON = BUY/HOLD** | **RISK_OFF = SELL/STAY OUT**

In [10]:
from scripts.s12_leveraged_execute import compute_live_signals

signals = compute_live_signals(
    data_dir=DATA_DIR,
    references=REFERENCES,
    vol_thresholds=VOL_THRESHOLDS,
)

Current Signals:
Ticker   Ref    Signal            Price     SMA200   RefVol  VolThresh
------------------------------------------------------------------------
   TECL     XLK    RISK_OFF     $  140.40 $  134.44   28.7%       22%
   NAIL     ITB    RISK_OFF     $  114.45 $  100.89   27.6%       22%


---
## Step 4: Generate Trade Plan

Decision logic based on current positions + signals:

| State | Signal | Action |
|-------|--------|--------|
| No positions | RISK_ON | BUY (50/50 from budget) |
| No positions | RISK_OFF | Wait |
| Holding | RISK_ON | Check drift, rebalance if >5% |
| Holding | RISK_OFF | SELL (exit to cash) |

In [11]:
from scripts.s12_leveraged_execute import generate_leveraged_trades

trades = generate_leveraged_trades(
    state=state,
    signals=signals,
    tickers=TICKERS,
    budget=BUDGET,
    target_weights=TARGET_WEIGHTS,
    ib_host=IB_HOST, ib_port=IB_PORT, ib_client_id=IB_CLIENT_ID,
)

if trades:
    pd.DataFrame(trades)

Error 10089, reqId 5: Requested market data requires additional subscription for API. See link in 'Market Data Connections' dialog for more details.Delayed market data is available.TECL ARCA/TOP/ALL, contract: Stock(conId=56539091, symbol='TECL', exchange='SMART', primaryExchange='ARCA', currency='USD', localSymbol='TECL', tradingClass='TECL')
Error 10089, reqId 6: Requested market data requires additional subscription for API. See link in 'Market Data Connections' dialog for more details.Delayed market data is available.NAIL ARCA/TOP/ALL, contract: Stock(conId=203435133, symbol='NAIL', exchange='SMART', primaryExchange='ARCA', currency='USD', localSymbol='NAIL', tradingClass='NAIL')



No positions and all signals RISK_OFF. Nothing to do.
Wait for SMA 200 crossover + vol filter to clear.


---
## Step 5: Execute Trades

**Set `CONFIRM = True` to place REAL orders on IB.**

Uses LIMIT GTC orders (submit anytime, fills at market open).  
**No trailing stops** — SMA 200 crossover is the exit signal.

In [12]:
# ============================================
CONFIRM = True  # Set True to execute
# ============================================

from scripts.s12_leveraged_execute import execute_leveraged_trades

exec_results = execute_leveraged_trades(
    trades, LIVE_DIR,
    ib_host=IB_HOST, ib_port=IB_PORT, ib_client_id=IB_CLIENT_ID + 1,
    use_limit_orders=USE_LIMIT_ORDERS,
    limit_buffer_pct=LIMIT_BUFFER_PCT,
    confirm=CONFIRM,
)

if exec_results:
    pd.DataFrame(exec_results)

No trades to execute.


---
## Step 6: Check Order Status

Run after market hours to verify fills. Read-only connection.

In [13]:
from scripts.s12_leveraged_execute import check_leveraged_status

check = check_leveraged_status(
    trades=trades,
    tickers=TICKERS,
    ib_host=IB_HOST, ib_port=IB_PORT, ib_client_id=IB_CLIENT_ID,
)

if check["filled"]:
    print("\nFILLED:")
    display(pd.DataFrame(check["filled"])[["action", "ticker", "shares", "fill_status"]])

if check["pending"]:
    print("\nPENDING:")
    display(pd.DataFrame(check["pending"])[["action", "ticker", "shares", "ib_status", "limit_price"]])

if check["missing"]:
    print("\nMISSING:")
    display(pd.DataFrame(check["missing"])[["action", "ticker", "shares", "current_position"]])

Connected (read-only): U9544585

Leveraged trades: 0
  Filled:  0
  Pending: 0
  Missing: 0

All leveraged trades completed. No action needed.


---
## Step 7: Fix Unfilled Orders

Cancels stale orders and resubmits with fresh market prices.

In [14]:
# ============================================
CONFIRM_FIX = False  # Set True to cancel + resubmit
# ============================================

from scripts.s12_leveraged_execute import fix_leveraged_orders

fix_results = fix_leveraged_orders(
    check, LIVE_DIR,
    ib_host=IB_HOST, ib_port=IB_PORT, ib_client_id=IB_CLIENT_ID + 1,
    use_limit_orders=USE_LIMIT_ORDERS,
    limit_buffer_pct=LIMIT_BUFFER_PCT,
    confirm=CONFIRM_FIX,
)

if fix_results:
    pd.DataFrame(fix_results)

Nothing to fix — all leveraged trades completed.


---

**Weekly workflow:**
1. Run Steps 1-3 to check positions and signals
2. Step 4 generates trades (if any needed)
3. Step 5 executes (set CONFIRM=True)
4. Next day: Steps 6-7 to verify fills

**Key rules:**
- RISK_ON = price above SMA 200 AND ref vol < 22%
- RISK_OFF = exit entirely (SMA crossover is the stop loss)
- NO trailing stops on leveraged ETFs
- Rebalance only if drift > 5% from 50/50 target